In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import dtale

In [2]:
pd.set_option('display.max_columns', None)
%matplotlib inline

Data Cleaning
-

In [3]:
def convert_dtype(x):
    if not x:
        return ''
    try:
        return str(x)
    except:
        return ''
def convert_dtypea(x):
    if not x:
        return ''
    try:
        return int(x)
    except:
        return ''
    
smile = pd.read_csv('../data/Smileon.2021.10.17.csv', skiprows=2, header=0, encoding='latin-1', converters={
     'LinkedToCallReportNum':convert_dtype,'ThirdPartyStateProvince':convert_dtype, 'FeedbackFromPhoneWorkerName':convert_dtype,
'SMILE ON 60+ Demographic Information - # of People in Household':convert_dtypea, 'Birth Year':convert_dtypea, 'SMILE ON 60+ Oral Care Encounter - 2393 - Quantity':convert_dtypea,
'SMILE ON 60+ Oral Care Encounter - 2950 - Quantity':convert_dtypea, 'SMILE ON 60+ Oral Care Encounter - 4342 - Quantity':convert_dtypea,
'SMILE ON 60+ Oral Care Encounter - 7140 - Quantity':convert_dtypea, 'SMILE ON 60+ Oral Care Encounter - 7320 - Quantity':convert_dtypea,
'SMILE ON 60+ Oral Care Encounter - Lab Assist narrative':convert_dtype, 'SMILE ON 60+ Case Management Information - What is the outcome for the client?':convert_dtype,
'SMILE ON 60+ Case Management Information - Dental Clinic':convert_dtype})

In [4]:
df = smile

In [5]:
df = df.drop(columns = 'SMILE ON 60+ Base-line Oral and Conditions Questions - Dental Clinic')
df = df.drop(columns = 'SMILE ON 60+ Case Management Information - Dental Clinic')
df = df.drop(columns = 'SMILE ON 60+ Presumptive Eligibility - What is the next step for clients encounter?')
df = df.drop(columns = 'SMILE ON 60+ Oral Care Encounter - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)')

In [6]:
##used this for general data cleaning
df.columns = [x.replace('SMILE ON 60+ Demographic Information -','') for x in df.columns]
df.columns = [x.replace(' - Quantity','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Screening -','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Base-line Oral and Conditions Questions - ','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Case Management Information - ','') for x in df.columns]
df.columns = [x.replace('(Select all that apply)','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Case Management Information - ','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Demographic Information - ','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Screening -','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Clinic Information - ','') for x in df.columns]

In [7]:
df_clean = df.drop(columns = ['ReportVersion','LinkedToCallReportNum','CallDateAndTimeEnd','CallLength',
            'PhoneWorkerNum','WasRealCall','WasHangup','WasSexCall','WasWrongNumber',
            'WasPrankCall','WasSilentCall','GeoCode','GeoAssignment','CensusDivision','CensusTrack',
            'CensusReportingArea','211Region','PhoneType','ThirdPartyName','ThirdPartyOrganization',
            'ThirdPartyPhoneNumber','ThirdPartyAddress','ThirdPartyCity','ThirdPartyCounty',
            'ThirdPartyStateProvince','ThirdPartyPostalCode','Narrative','VolunteerComments',
            'Feedback','CallersFeedback','TextField2','TextField3','TextField4','TextField5',
            'TextField6','TextField7','TextField8','TextField9','TextField10','EnteredByWorkerNum',
            'EnteredOn','Supervisor','Reviewed','FeedbackStatus','FeedbackFromPhoneWorkerNum',
             'FeedbackFromPhoneWorkerName','ReferralsMade','IPAddress','OrgNum','VolIdentifier',
            'SMILE ON 60+ Verification of Eligibility - Verification Date',
            'Did the client commit to referral?','Did you help in these other areas?',
             'Transportation Barriers','Action: Upload Before Picture on Finish Tab','Was initial appointment made?',
            'Was initial transportation assistance or referral provided?',
            'SMILE ON 60+ Oral Care Encounter - Action: Upload After Picture on Finish Tab',
            'SMILE ON 60+ Presumptive Eligibility - Dental Insurance?','SMILE ON 60+ Presumptive Eligibility - Do they pre-qualify for SMILE ON 60+?',
            'SMILE ON 60+ Site Information - Enroller ID',
            'SMILE ON 60+ Site Information - Enroller Location',
            'SMILE ON 60+ Verification of Eligibility - Transportation Arranged','SMILE ON 60+ Verification of Eligibility - Verification of Eligibility',
            'SMILE ON 60+ Treatment Plan - Action: Upload After Picture on Finish Tab',' Root Fragments',' Need for Periodontal Care',' Suspicious Soft Tissue Lesions',
            ' Untreated Decay',' What other assistance programs are you on?',' Preferred Language/Language Needed',
            ' How did you first hear about the SMILE ON 60+ program?','SMILE ON 60+ Presumptive Eligibility - What is the next step for clients encounter? ',
            'What is the next step for clients encounter?'])

In [8]:
df_clean.columns = [x.replace('SMILE ON 60+ Oral Care Encounter -','') for x in df_clean.columns]
df_clean.columns = [x.replace('Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)','social_6_restored') for x in df_clean.columns]
df_clean.columns = [x.replace('SMILE ON 60+ Presumptive Eligibility -','') for x in df_clean.columns]
df_clean.columns = [x.replace('What barriers have kept you from finding a consistent dental home?','barriers') for x in df_clean.columns]
df_clean.columns = [x.replace('How would you describe the condition of your mouth and teeth?','mouth_condition') for x in df_clean.columns]
df_clean.columns = [x.replace('Have you ever felt that the appearance of your mouth and teeth affected your quality of life?','teeth_affect_qualifty_of_life') for x in df_clean.columns]
df_clean.columns = [x.replace('No Code - ','') for x in df_clean.columns]
df_clean.columns = [x.replace('Experienced any problems related to mouth/teeth during past 12 months?','problems_past_year') for x in df_clean.columns]
df_clean.columns = [x.replace('How often do you brush your teeth?','brush_frequency') for x in df_clean.columns]


In [9]:
df_clean = df_clean[df_clean.CallerNum >= 0]

Subgroup the Data by Treatment Urgency Column. Merged to keep all records for patients labeled in any  3 subgroups at any point
-
Urgent Care - within next week-pain or infection

No obvious problem

Early care-within next several weeks


In [10]:
urgent = df_clean.loc[df_clean[' Treatment Urgency'] == 'Urgent Care - within next week-pain or infection']
urgent = urgent.filter(['CallReportNum','CallerNum'])

In [11]:
no_problem = df_clean.loc[df_clean[' Treatment Urgency'] == 'No obvious problem']
no_problem = no_problem.filter(['CallReportNum','CallerNum'])

In [12]:
early_care = df_clean.loc[df_clean[' Treatment Urgency'] == 'Early care-within next several weeks']
early_care = early_care.filter(['CallReportNum','CallerNum'])

In [13]:
urgent_merged = df_clean.merge(urgent,how='inner',on='CallerNum')

In [14]:
no_problem_merged = df_clean.merge(no_problem,how='inner',on='CallerNum')

In [15]:
early_care_merged = df_clean.merge(early_care,how='inner',on='CallerNum')

Create dummy variables for; was treatment plan develped,is treatment plan completed, was social 6 restored
-
use these to quantify number of completed treatment plans vs on-going treatment plans

In [16]:
created_dummy = pd.get_dummies(df_clean['SMILE ON 60+ Treatment Plan - Was a treatment plan developed?'])
created_dummy = created_dummy.rename(columns = {'No':'not_created','Yes':'created'}).astype(int)

In [17]:
completed_dummy = pd.get_dummies(df_clean['SMILE ON 60+ Treatment Plan - Is the treatment plan completed?'])
completed_dummy = completed_dummy.rename(columns = {'No':'not_completed','Yes':'completed'}).astype(int)

In [18]:
social_dummy = pd.get_dummies(df_clean['SMILE ON 60+ Treatment Plan - social_6_restored'])
social_dummy = social_dummy.rename(columns = {'No':'not_restored','Yes':'restored'}).astype(int)

In [19]:
df_clean = pd.concat([df_clean,created_dummy],axis=1)

In [20]:
df_clean = pd.concat([df_clean,completed_dummy],axis=1)

In [21]:
df_clean = pd.concat([df_clean,social_dummy],axis=1)

In [22]:
df_grouped = df_clean.groupby('CallerNum').agg('sum')

In [23]:
#difference between incomplete and created shows unfinished/unsuccessful treatment plans
df_grouped['incomplete'] = df_grouped['created'] - df_grouped['completed']

In [24]:
df_grouped.incomplete.value_counts()

 0     5201
 1     1810
 2      990
 3      731
 4      508
 5      363
 6      274
 7      216
-1      177
 8      121
 9       76
 10      75
 11      42
 12      35
-2       29
 13      20
 14       8
-3        8
 16       5
 15       5
-4        4
 19       2
 18       2
 23       2
 20       1
 21       1
-8        1
 30       1
-5        1
Name: incomplete, dtype: int64

apply process to each of 3 subgroups
-

Urgent

In [25]:
urgent_created_dummy = pd.get_dummies(urgent_merged['SMILE ON 60+ Treatment Plan - Was a treatment plan developed?'])
urgent_created_dummy = urgent_created_dummy.rename(columns = {'No':'not_created','Yes':'created'}).astype(int)
urgent_completed_dummy = pd.get_dummies(urgent_merged['SMILE ON 60+ Treatment Plan - Is the treatment plan completed?'])
urgent_completed_dummy = urgent_completed_dummy.rename(columns = {'No':'not_completed','Yes':'completed'}).astype(int)
urgent_social_dummy = pd.get_dummies(urgent_merged['SMILE ON 60+ Treatment Plan - social_6_restored'])
urgent_social_dummy = urgent_social_dummy.rename(columns = {'No':'not_restored','Yes':'restored'}).astype(int)
urgent_merged = pd.concat([urgent_merged,urgent_created_dummy],axis=1)
urgent_merged = pd.concat([urgent_merged,urgent_completed_dummy],axis=1)
urgent_merged = pd.concat([urgent_merged,urgent_social_dummy],axis=1)

In [26]:
urgent_grouped = urgent_merged.groupby('CallerNum').agg('sum')

In [27]:
#difference between incomplete and created shows unfinished/unsuccessful treatment plans
urgent_grouped['incomplete'] = urgent_grouped['created'] - urgent_grouped['completed']

In [28]:
urgent_grouped.incomplete.value_counts()

 1     105
 0      82
 2      61
 3      50
 4      31
 6      28
 5      26
 7      24
 10     15
 8      12
 11      9
 9       9
-1       8
 12      7
 13      4
-2       3
 15      2
 19      2
 14      1
 16      1
 20      1
 24      1
 22      1
Name: incomplete, dtype: int64

no problem

In [29]:
no_problem_created_dummy = pd.get_dummies(no_problem_merged['SMILE ON 60+ Treatment Plan - Was a treatment plan developed?'])
no_problem_created_dummy = no_problem_created_dummy.rename(columns = {'No':'not_created','Yes':'created'}).astype(int)
no_problem_completed_dummy = pd.get_dummies(no_problem_merged['SMILE ON 60+ Treatment Plan - Is the treatment plan completed?'])
no_problem_completed_dummy = no_problem_completed_dummy.rename(columns = {'No':'not_completed','Yes':'completed'}).astype(int)
no_problem_social_dummy = pd.get_dummies(no_problem_merged['SMILE ON 60+ Treatment Plan - social_6_restored'])
no_problem_social_dummy = no_problem_social_dummy.rename(columns = {'No':'not_restored','Yes':'restored'}).astype(int)
no_problem_merged = pd.concat([no_problem_merged,no_problem_created_dummy],axis=1)
no_problem_merged = pd.concat([no_problem_merged,no_problem_completed_dummy],axis=1)
no_problem_merged = pd.concat([no_problem_merged,no_problem_social_dummy],axis=1)

In [30]:
no_problem_grouped = no_problem_merged.groupby('CallerNum').agg('sum')

In [31]:
#difference between incomplete and created shows unfinished/unsuccessful treatment plans
no_problem_grouped['incomplete'] = no_problem_grouped['created'] - no_problem_grouped['completed']

In [32]:
no_problem_grouped.incomplete.value_counts()

 0     237
 1     223
 2     175
 3      87
 4      79
 6      61
 5      49
-1      46
 8      31
 7      29
 12     20
 10     15
-2      13
 9      10
 11      8
 13      6
 20      6
 16      5
-4       4
 14      4
 21      3
 18      2
-3       2
-6       2
 15      2
 19      1
 24      1
 36      1
-10      1
-12      1
Name: incomplete, dtype: int64

Early care

In [34]:
early_created_dummy = pd.get_dummies(early_care_merged['SMILE ON 60+ Treatment Plan - Was a treatment plan developed?'])
early_created_dummy = early_created_dummy.rename(columns = {'No':'not_created','Yes':'created'}).astype(int)
early_completed_dummy = pd.get_dummies(early_care_merged['SMILE ON 60+ Treatment Plan - Is the treatment plan completed?'])
early_completed_dummy = early_completed_dummy.rename(columns = {'No':'not_completed','Yes':'completed'}).astype(int)
early_social_dummy = pd.get_dummies(early_care_merged['SMILE ON 60+ Treatment Plan - social_6_restored'])
early_social_dummy = early_social_dummy.rename(columns = {'No':'not_restored','Yes':'restored'}).astype(int)
early_care_merged = pd.concat([early_care_merged,early_created_dummy],axis=1)
early_care_merged = pd.concat([early_care_merged,early_completed_dummy],axis=1)
early_care_merged = pd.concat([early_care_merged,early_social_dummy],axis=1)

In [35]:
early_care_grouped = early_care_merged.groupby('CallerNum').agg('sum')

In [36]:
#difference between incomplete and created shows unfinished/unsuccessful treatment plans
early_care_grouped['incomplete'] = early_care_grouped['created'] - early_care_grouped['completed']

In [40]:
early_care_grouped.incomplete.value_counts()

 1     998
 2     588
 0     558
 3     387
 4     356
 6     243
 5     192
 8     137
 7     114
 10     76
-1      72
 9      61
 12     53
 14     29
-2      21
 11     19
 18     18
 16     18
 20     17
 15     11
 21     11
 24      9
 22      8
 13      7
-3       6
 28      5
 36      4
 26      3
 33      3
 30      2
 19      2
-4       2
 27      2
 60      1
 48      1
 46      1
 42      1
 40      1
 32      1
-8       1
-6       1
 39      1
 69      1
Name: incomplete, dtype: int64

In [54]:
early_care_merged.loc[early_care_merged['CallerNum'] == 2414923]

,CallReportNum_x,CallDateAndTimeStart,CallerNum,CityName,CountyName,StateProvince,CountryName,PostalCode,# of People in Household,Birth Year,Other Language,SMILE ON 60+ Notes (Optional) - Notes,1351,1354,2140,2150,2160,2161,220,230,2330,2331,2332,2335,2390,2391,2392,2393,2394,2915,2920,2933,2940,2950,2954,2955,2980,3220,4341,4342,6930,6980,7111,7140,7210,7220,7230,7240,7241,7250,7261,7285,7286,7310,7311,7320,7321,7410,7411,7412,7413,7414,7415,7440,7450,7451,7460,7461,7471,7510,7511,7970 - Mandibular Quantity,7970 - Maxillary Quantity,7971,7972,9110,9951,Lab Assist narrative,Denture Step Mandibular Quantity,Denture Step Maxillary Quantity,Last Screening Date,Are you limited in what you can eat?,Do you need help in these other areas?,teeth_affect_qualifty_of_life,mouth_condition,What is the next step for client encounter?,barriers,What is the outcome for the client?,Is patient of record date if yes last seen?,Gender,Household Income Level,Race/Ethnicity,When is the last time you visited your dentist?,Is the treatment plan completed?,Was a treatment plan developed?,What care was provided to the enrollee?,# of Lower Natural Teeth,# of Upper Natural Teeth,Do you have removable dentures?,Do you usually wear your denture(s) when you eat?,problems_past_year,brush_frequency,Treatment Urgency,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,SMILE ON 60+ Treatment Plan - social_6_restored,CallReportNum_y,not_created,created,not_completed,completed,not_restored,restored
9850,91415863,8/10/2021 11:39,2414923,Nashville,Davidson,TN,United States,37211.0,1,1956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,2/22/2021,NaN,NaN,NaN,NaN,NaN,Dental cost,,NaN,Male,NaN,Caucasian,Less than 12 months,NaN,NaN,2392 - Posterior two surface resin - 15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,No,84699994,0,1,1,0,1,0
9851,90086522,7/6/2021 12:31,2414923,Nashville,Davidson,TN,United States,37211.0,1,1956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,2/22/2021,NaN,NaN,NaN,NaN,NaN,Dental cost,,NaN,Male,NaN,Caucasian,Less than 12 months,NaN,NaN,180 - Comprehensive Periodontal Evaluation - 1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,Yes,84699994,0,1,1,0,0,1
9852,86612319,3/30/2021 11:43,2414923,Nashville,Davidson,TN,United States,37211.0,1,1956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,2/22/2021,NaN,NaN,NaN,NaN,NaN,Dental cost,,NaN,Male,NaN,Caucasian,Less than 12 months,NaN,NaN,140 - Limited Oral evaluation - problem focuse...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,No,84699994,0,1,1,0,1,0
9853,85496705,2/22/2021 9:45,2414923,Nashville,Davidson,TN,United States,37211.0,1,1956,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,2/22/2021,NaN,NaN,NaN,NaN,NaN,Dental cost,,NaN,Male,NaN,Caucasian,Less than 12 months,NaN,NaN,1354 - Interim caries arresting medicament app...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Yes,No,84699994,0,1,1,0,1,0
9854,84699994,2/11/2021 9:40,2414923,Nashville,Davidson,TN,United States,37211.0,1,1956,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,

In [57]:
d = dtale.show(df_clean)
d.open_browser()